In [1]:
import tweepy
import pandas
import csv
import re #regular expression
import nltk
from nltk.corpus import stopwords
##nltk.download('stopwords')# Download stopwords
import textblob
from textblob import TextBlob
import sys
import psycopg2
import time
import numpy as np
import json

In [2]:
consumer_key = '1QRIUdf4Sx4xdMDc46nQ6vYZ3'
consumer_secret = '5tEVf8mm8dUjYftXiae9QERBDFiKXrVSWkjRm2jpfBNy0NRm1Q'
access_token = '1191719379186192384-Gn0pBMspYGX2BCQxqyZACjNtAQ67YL'
access_secret = 'WuVwxRsXdw9gtTK1OTWK7McCtAC7l8Y0BSfYEUhcYEWv0'
access_token_secret=access_secret

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
##print(x.translate(non_bmp_map))

In [3]:
df = pandas.read_csv('managers_world_names.csv', sep=';', na_values="(missing)")
df.shape[0]
names=[]
for jj in range(df.shape[0]):
    names.append(df['names'][jj])

In [4]:
def cleaning(tweet):
    ##Handle Emoticons and Emojis
    #HappyEmoticons
    emoticons_happy = set([
        ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
        ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
        '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
        'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
        '<3'
        ])
    
    #Sad Emoticons
    emoticons_sad = set([
        ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
        ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
        ':c', ':{', '>:\\', ';('
        ])
    
    ##Emoji patterns
    #combine sad and happy emoticons
    emoticons = emoticons_happy.union(emoticons_sad)
    
    emoji_pattern = re.compile("["
             u"\U0001F600-\U0001F64F"  # emoticons
             u"\U0001F300-\U0001F5FF"  # symbols & pictographs
             u"\U0001F680-\U0001F6FF"  # transport & map symbols
             u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
             u"\U00002702-\U000027B0"
             u"\U000024C2-\U0001F251"
             "]+", flags=re.UNICODE)

##    extract_link
##    regex = r' https?://[^\s<>"]+|www\.[^\s<>"]+ '
##    match = re.search(regex, tweet.full_text)
##    if match:
##        print("links: ", match)

    #replace consecutive non-ASCII characters with a space
    tweet1 = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', tweet)
##    print("a1             ",tweet1)

    #remove emojis from tweet
    tweet2 = emoji_pattern.sub(r'', tweet1)   
##    print("a2             ",tweet2)

    #utility function to clean tweet text by removing links, special characters using simple regex statements. 
    tweet3=re.sub("(@[A-Za-z0-9_]+)|([^0-9A-Za-z_ \t]) |(\w+:\/\/\S+)", " ", tweet2) 
##    print("a3             ",tweet3)

    #removing numbers
    tweet4= re.sub(" (\d+) | (\d+)(.)(\d+) ", ' ',tweet3) # number alone
    tweet5= re.sub("(\d+)", ' ',tweet4) # number attached
##    print("a4             ",tweet5)

    # removing some charachters 
####    tweet6= re.sub(".", ' ',tweet4) # !!!! remove all
    tweet6= re.sub("[\";:\|,~!=\-+#$%&*\(\)?\.\/]+", ' ',tweet5)
##    print("a5             ",tweet6)
   
    # Create a sublist of lower case words for each tweet
    words_in_tweet = tweet6.lower().split()
##    print("a5    ",words_in_tweet)

    stop_words = set(stopwords.words('english'))
    tweet_cleaned=''
    for word in words_in_tweet:
        if word not in stop_words:
            tweet_cleaned +=' '+ word       
##    print("a6    ",tweets_cleaned)

    return tweet_cleaned

In [5]:
'''
csvFile_Loc = open('manager_tweets1.csv', 'w',newline='')
csvWriter = csv.writer(csvFile_Loc,delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
csvWriter.writerow(["screen_name",
                    "user_name",
                    "user_id",
                    "tweet",
                    "cleaned_tweet",
                    "lang",
                    "tweet_created_at",
                    "location_profile",
                    "description_profile",
                    "all_tweet"])
'''

'\ncsvFile_Loc = open(\'manager_tweets1.csv\', \'w\',newline=\'\')\ncsvWriter = csv.writer(csvFile_Loc,delimiter=\';\', quotechar=\'"\', quoting=csv.QUOTE_MINIMAL)\ncsvWriter.writerow(["screen_name",\n                    "user_name",\n                    "user_id",\n                    "tweet",\n                    "cleaned_tweet",\n                    "lang",\n                    "tweet_created_at",\n                    "location_profile",\n                    "description_profile",\n                    "all_tweet"])\n'

In [6]:
'''
for tweet in tweepy.Cursor(api.user_timeline,user_name='MansouriJafar').items():
    print (tweet)
'''

'''
n=0
for tweet in tweepy.Cursor(api.user_timeline,id='USATODAY',since="2019-11-19",
                   until="2019-11-20").items():
    n +=1
    #print (tweet)
'''

'''
a=(5,1)
n=np.zeros(a)
j=0
names=['BillGates','richardbranson','Louis_Tomlinson','TheEllenShow','realDonaldTrump']
for i in range(5):
    n[i]=0
    for tweet in tweepy.Cursor(api.user_timeline,screen_name=names[i]).items():
        n[i] +=1
        j +=1
        #print (tweet)
        print(j)
'''
with open('manager_tweets1.json', 'w') as f:  # writing JSON object
    json.dump(1, f)

In [7]:
#names=['BillGates','richardbranson','Louis_Tomlinson','TheEllenShow','realDonaldTrump','melindagates']
n=np.zeros(len(names))


In [ ]:
     
for i in range(988,len(names)):
    
    consumer_key = '1QRIUdf4Sx4xdMDc46nQ6vYZ3'
    consumer_secret = '5tEVf8mm8dUjYftXiae9QERBDFiKXrVSWkjRm2jpfBNy0NRm1Q'
    access_token = '1191719379186192384-Gn0pBMspYGX2BCQxqyZACjNtAQ67YL'
    access_secret = 'WuVwxRsXdw9gtTK1OTWK7McCtAC7l8Y0BSfYEUhcYEWv0'
    access_token_secret=access_secret

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True)

    
    try:
        for tweet in tweepy.Cursor(api.user_timeline,screen_name=names[i],tweet_mode="extended" ).items():
         #   print (tweet)
            n[i] +=1
            data= [tweet.user.screen_name.translate(non_bmp_map),
                   tweet.user.name.translate(non_bmp_map),
                   tweet.user.id,
                   tweet.full_text.translate(non_bmp_map),
                   cleaning(tweet.full_text),
                   tweet.lang,
                   tweet.created_at,
                   tweet.user.location.translate(non_bmp_map),
                   tweet.user.description,
                   tweet._json]
            csvFile_Loc = open('manager_tweets1.csv', 'a',encoding="utf-8",newline='')
            csvWriter = csv.writer(csvFile_Loc, delimiter=';' , quotechar='"',  quoting=csv.QUOTE_MINIMAL)
            csvWriter.writerow(data)


            data1= [tweet.user.screen_name.translate(non_bmp_map),
                   tweet.user.name.translate(non_bmp_map),
                   tweet.user.id,
                   tweet.full_text.translate(non_bmp_map),
                   cleaning(tweet.full_text),
                   tweet.lang,
                   tweet.created_at,
                   tweet.user.location.translate(non_bmp_map),
                   tweet.user.description,
                   str(tweet._json)]

            conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=Jafarsql")
            cur = conn.cursor()
            cur.execute("INSERT INTO manager_tweets1 VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",data1);
            conn.commit()


            with open('manager_tweets1.json', 'a') as f:  # writing JSON object
                json.dump(tweet._json, f)
    except:
        print("An exception occurred")
        print(i)
        
        
    if i%4 == 3: 
        print('artificial break')
        time.sleep(60*10)

An exception occurred
415
artificial break
artificial break
artificial break
artificial break
An exception occurred
430
artificial break
artificial break
An exception occurred
436
artificial break
artificial break
artificial break
An exception occurred
449
An exception occurred
450
artificial break
artificial break
artificial break
artificial break
artificial break
artificial break
artificial break
artificial break
artificial break
artificial break
An exception occurred
490
An exception occurred
491
artificial break
artificial break
artificial break
An exception occurred
500
An exception occurred
501
artificial break
artificial break
An exception occurred
511
artificial break
artificial break
artificial break
artificial break
artificial break
An exception occurred
528
artificial break
artificial break
artificial break
An exception occurred
543
artificial break
artificial break
artificial break
artificial break
An exception occurred
557
artificial break
artificial break
artificial break

In [33]:
n[128]

0.0